In [2]:
import pandas as pd
import numpy as np
quandl_api_key = '_umNYuQHdkCgs9Rcm4Fv'

In [18]:
df_es = pd.read_csv('/Users/kush/Desktop/EP.csv', sep='\t')
df_nq = pd.read_csv('/Users/kush/Desktop/ENQ.csv', sep='\t')

In [19]:
df_es

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>
0,1997.09.08,15:40:00,934.00,934.00,934.00,934.00,1,0,0
1,1997.09.09,10:30:00,933.75,934.50,933.75,933.75,151,0,0
2,1997.09.09,10:31:00,933.75,934.25,933.75,934.25,51,0,0
3,1997.09.09,10:32:00,934.25,934.75,934.25,934.25,101,0,0
4,1997.09.09,10:33:00,934.50,934.50,934.50,934.50,1,0,0
...,...,...,...,...,...,...,...,...,...
7489280,2023.04.14,20:55:00,4173.50,4173.50,4172.50,4172.50,178,287,25
7489281,2023.04.14,20:56:00,4172.75,4172.75,4171.75,4171.75,250,3246,25
7489282,2023.04.14,20:57:00,4171.75,4172.00,4171.50,4171.50,138,378,25
7489283,2023.04.14,20:58:00,4171.75,4171.75,4171.25,4171.50,187,3123,25


In [3]:
###### S&P ######

# Combine the date and time columns into a single datetime column
df_es['datetime'] = pd.to_datetime(df_es['<DATE>'] + ' ' + df_es['<TIME>'])

# Drop the unnecessary columns
df_es = df_es.drop(columns=['<DATE>', '<TIME>', '<TICKVOL>', '<VOL>', '<SPREAD>'])

# Rename columns to remove symbols and make them lowercase
df_es.columns = [col.replace('<', '').replace('>', '').lower() for col in df_es.columns]
df_es = df_es[['datetime', 'open', 'high', 'low', 'close']]

# Display the updated dataframe
df_es.head()

###### NASDAQ ######

# Combine the date and time columns into a single datetime column
df_nq['datetime'] = pd.to_datetime(df_nq['<DATE>'] + ' ' + df_nq['<TIME>'])

# Drop the unnecessary columns
df_nq = df_nq.drop(columns=['<DATE>', '<TIME>', '<TICKVOL>', '<VOL>', '<SPREAD>'])

# Rename columns to remove symbols and make them lowercase
df_nq.columns = [col.replace('<', '').replace('>', '').lower() for col in df_nq.columns]
df_nq = df_nq[['datetime', 'open', 'high', 'low', 'close']]

# Display the updated dataframe
#df_nq.head()


In [4]:
# Combine the two datasets based on the datetime column
combined_data = pd.merge(df_es, df_nq, on='datetime', suffixes=('_es', '_nq'))

# Convert the datetime column to a datetime object
combined_data['datetime'] = pd.to_datetime(combined_data['datetime'])

In [5]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

# Select columns 2-10 for normalization
norm_cols = combined_data.iloc[:, 1:10].columns.tolist()

# Normalize the selected columns
scaler = MinMaxScaler()
combined_data[norm_cols] = scaler.fit_transform(combined_data[norm_cols])

In [6]:
combined_data.set_index('datetime', inplace=True)

In [18]:
import numpy as np

timeframes = {'15min' : 100, '5min' : 300, '1min' : 1500}

def data_generator(df, timeframes, batch_size):
    """Generates resampled data based on the specified timeframes and outputs data in the format of X and y.

    Args:
        df (pandas.DataFrame): The input dataframe on the 1 min timeframe.
        timeframes (dict): The dictionary of timeframes and their corresponding number of timepoints.
        batch_size (int): The number of samples to generate per batch.

    Returns:
        tuple: A tuple of the X and y arrays in the format of (X, y).
    """
    while True:
        # Initialize X and y arrays
        X_15min = []
        X_5min = []
        X_1min = []
        y = []

        # Resample the data for each timeframe from the beginning
        resampled_data = {}
        for timeframe, num_points in timeframes.items():
            resampled_data[timeframe] = df.resample(timeframe).apply({'open_es': 'first', 'high_es': 'max', 'low_es': 'min', 'close_es': 'last', 'open_nq': 'first', 'high_nq': 'max', 'low_nq': 'min', 'close_nq': 'last'}).fillna(method='ffill')

        # Iterate through each day at 9:30am
        for date, _ in df.resample('B'):
            # Get the data for the appropriate time intervals for each timeframe for that day at 9:30am
            end_time = pd.Timestamp(date) + pd.Timedelta(hours=9, minutes=30)
            data = {}
            skip_date = False
            #every timeframe takes data from the same interval of 1500 minutes back, just at different TFs.
            # 1499 instead of 1500 bc if we use 1500, it will include another interval, increasing the timepoint by an additional value
            start_time = end_time - pd.Timedelta(minutes=1499) 
            for timeframe, num_points in timeframes.items():    
                data[timeframe] = resampled_data[timeframe].loc[start_time:end_time]
                if len(data[timeframe]) < num_points:
                    skip_date = True
                    break            

            if skip_date:
                continue

            # Combine the resampled data for each timeframe into a single array
            resampled_data_combined = [data[timeframe].values for timeframe in timeframes]

            # Get the highest high and lowest low between 9:30am and 12 noon for that day
            high = df.loc[end_time:end_time+pd.Timedelta(minutes=150)].high_es.max()
            low = df.loc[end_time:end_time+pd.Timedelta(minutes=150)].low_es.min()

            # Add the resampled data and high/low values to the X and y arrays
            X_15min.append(resampled_data_combined[0])
            X_5min.append(resampled_data_combined[1])
            X_1min.append(resampled_data_combined[2])
            y.append((high, low))

            # Generate a new batch if the current batch is full
            if len(X_1min) == batch_size:
                yield ([np.array(X_15min), np.array(X_5min), np.array(X_1min)], np.array(y))
                X_15min = []
                X_5min = []
                X_1min = []
                y = []

        # Yield the final batch if there are any remaining samples
        #if len(X_1min) > 0:
        #    yield ([np.array(X_15min), np.array(X_5min), np.array(X_1min)], np.array(y))

In [19]:
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform

# Define the input layers
input_15min = Input(shape=(100, 8, 1), name='15min')
input_5min = Input(shape=(300, 8, 1), name='5min')
input_1min = Input(shape=(1500, 8, 1), name='1min')

# Define the convolutional layers with glorot_uniform initialization
conv_15min = Conv2D(filters=16, kernel_size=(5, 1), activation='relu', kernel_initializer=glorot_uniform(seed=1))(input_15min)
conv_5min = Conv2D(filters=16, kernel_size=(15, 1), activation='relu', kernel_initializer=glorot_uniform(seed=2))(input_5min)
conv_1min = Conv2D(filters=16, kernel_size=(50, 1), activation='relu', kernel_initializer=glorot_uniform(seed=3))(input_1min)

# Flatten the convolutional layers
flatten_15min = Flatten()(conv_15min)
flatten_5min = Flatten()(conv_5min)
flatten_1min = Flatten()(conv_1min)

# Concatenate the flattened layers
concatenated = Concatenate()([flatten_15min, flatten_5min, flatten_1min])

# Add a dense layer
dense_layer = Dense(32, activation='relu', kernel_initializer=glorot_uniform(seed=4))(concatenated)

# Output layer
output_layer = Dense(2, activation='linear', name='output', kernel_initializer=glorot_uniform(seed=5))(dense_layer)

# Define the model
model = Model(inputs=[input_15min, input_5min, input_1min], outputs=output_layer)


In [9]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from keras import optimizers
import tensorflow as tf
import math

# Define the number of epochs
epochs = 2

# Compile the model with a learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Define the batch size
batch_size = 32

# Create the data generator
train_generator = data_generator(combined_data.loc['2012':'2020'], timeframes, batch_size)
val_generator = data_generator(combined_data.loc['2021':], timeframes, batch_size)

# Define the early stopping criteria
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

# Train the model using the generator
history = model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch= 2348 // batch_size,
    validation_data=val_generator,
    validation_steps= 858 // batch_size,
    verbose=1,
    callbacks=[early_stopping]
)


Epoch 1/2


2023-04-17 15:55:06.478293: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


73/73 [==============================] - 10s 138ms/step - loss: nan - val_loss: nan
Epoch 2/2
73/73 [==============================] - 10s 133ms/step - loss: nan - val_loss: nan


In [ ]:
import tensorflow as tf
tf.keras.utils.plot_model(model, show_shapes=True)

In [70]:
test_gen = data_generator(combined_data.loc['2012':'2020'], timeframes, 1)

for i in range(1):
    X, y = next(test_gen)

In [ ]:
# Create a new model that outputs the output of each layer
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)

# Get the output of each layer for a sample input
sample_input = X
activations = activation_model.predict(sample_input)
print(y)

# Print the output of each layer
for i, activation in enumerate(activations):
    print(f'Layer {i}:')
    print(activation.mean())

In [17]:
for layer in model.layers:
    print("===== LAYER: ", layer.name, " =====")
    if layer.get_weights() != []:
        weights = layer.get_weights()[0].mean()
        biases = layer.get_weights()[1]
        print("weights:")
        print(weights)
        print("biases:")
        print(biases)
    else:
        print("weights: ", [])

===== LAYER:  15min  =====
weights:  []
===== LAYER:  5min  =====
weights:  []
===== LAYER:  1min  =====
weights:  []
===== LAYER:  conv2d_3  =====
weights:
nan
biases:
[nan  0. nan nan  0.  0. nan  0.  0. nan nan  0.  0. nan nan nan]
===== LAYER:  conv2d_4  =====
weights:
nan
biases:
[ 0.  0. nan  0.  0.  0. nan  0.  0. nan nan  0.  0.  0.  0. nan]
===== LAYER:  conv2d_5  =====
weights:
nan
biases:
[nan nan nan nan nan nan  0. nan nan  0.  0. nan  0.  0.  0.  0.]
===== LAYER:  flatten_3  =====
weights:  []
===== LAYER:  flatten_4  =====
weights:  []
===== LAYER:  flatten_5  =====
weights:  []
===== LAYER:  concatenate_1  =====
weights:  []
===== LAYER:  dense_1  =====
weights:
nan
biases:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
===== LAYER:  output  =====
weights:
nan
biases:
[nan nan]


In [20]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)
loss_values = []

for step in range(10):
    # Generate a batch of data
    X_batch, y_batch = next(train_generator)

    # Train the model on the batch of data
    loss = model.train_on_batch(X_batch, y_batch)

    # Append the loss value to the loss_values array
    loss_values.append(loss)

loss_values

[0.039047010242938995, nan, nan, nan, nan, nan, nan, nan, nan, nan]